In [22]:
import requests
from bs4 import BeautifulSoup
import re 
import random

original_url = 'https://repo1.maven.org/maven2/'

response = requests.get(original_url)

# version folder pattern
pattern = r'^\d+\.\d+(\.\d+)?(-[a-zA-Z0-9]+)?$'


def traverse_folders(url):
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception
    
    current_result = traverse_links(response)
    if current_result[1]:
        # version folder has been reached, check for classifier
        response = requests.get(f"{url}{current_result[0]}")
        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find_all('a', href=True)
        for i, link in enumerate(links, 1):
            if "sources" in link.get_text():
                return f"{url}{current_result[0]}sources".replace(original_url, "")
        return f'{url}{current_result[0][:-1]}:" "'.replace(original_url, "")
    return traverse_folders(f"{url}{current_result[0]}")

def traverse_links(response):
    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.find_all('a', href=True)
    link = list(links)[random.randrange(1, len(links))]

    while link.get_text() == "../":
        link = list(links)[random.randrange(1, len(links))]

    result = re.match(pattern, link.get_text().replace("/", ""))
    if result:
        # version folders was found
        return (link.get_text(), True)
    else:
        print("No version found")
        print(link.get_text())
        return (link.get_text(), False)

def replace_last_two_slashes(s):
    parts = s.rsplit('/', 2)
    
    return ':'.join(parts)
print()

In [24]:
import subprocess
import os

# Define the command as a list of strings
command = [
    "java",
    f"-Dartifacts={replace_last_two_slashes(traverse_folders(original_url))}",
    "-Dindexer.work.path=lasso_crawler",
    "-Dbatch.maven.repo.url=http://localhost:8081/repository/maven-public/",
    "-Dlasso.indexer.worker.threads=1",
    "-jar", "/Users/alexanderberndt/Documents/lasso/crawler/target/crawler-1.0.0-SNAPSHOT.jar"
]
print(" ".join(command))

No version found
ognl/
No version found
ognl/
java -Dartifacts=ognl/ognl:3.0.8:sources -Dindexer.work.path=lasso_crawler -Dbatch.maven.repo.url=http://localhost:8081/repository/maven-public/ -Dlasso.indexer.worker.threads=1 -jar /Users/alexanderberndt/Documents/lasso/crawler/target/crawler-1.0.0-SNAPSHOT.jar


In [ ]:
print("Output:", result.stdout)
print("Error:", result.stderr)